In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
pd.options.display.float_format = '{:.5f}'.format
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/housing-lang-detect/housing_lang.csv


In [ ]:
!pip install openpyxl

In [4]:
data = pd.read_csv('/kaggle/input/housing-lang-detect/housing_lang.csv')

In [6]:
data.head()

,Unnamed: 0,adAvailableDate,adDescription,adLangDetected
0,0,not available,Appartement avec beaucoup de charme à louer à ...,fr
1,1,nach Vereinbarung,Die Highlights:<br />» Gut geschnittener Büror...,de
2,2,sofort,Für diese Wohnung suchen wir ein nettes Paar (...,de
3,3,sofort,<h3>Chiasso</h3><p><p>Si affitta in pieno a ce...,it
4,4,sofort,"Attico mansardato, posto in elegante palazzina...",it


In [7]:
data = data.drop('Unnamed: 0', axis=1)
data.head()

,adAvailableDate,adDescription,adLangDetected
0,not available,Appartement avec beaucoup de charme à louer à ...,fr
1,nach Vereinbarung,Die Highlights:<br />» Gut geschnittener Büror...,de
2,sofort,Für diese Wohnung suchen wir ein nettes Paar (...,de
3,sofort,<h3>Chiasso</h3><p><p>Si affitta in pieno a ce...,it
4,sofort,"Attico mansardato, posto in elegante palazzina...",it


# Installing library for detecting language

In [8]:
!pip install langdetect

     |████████████████████████████████| 981 kB 619 kB/s            
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993242 sha256=0c9b955b3945734193065e721054444e88630ee22428f7ee5ca22e70d2c8eb2b
  Stored in directory: /root/.cache/pip/wheels/c5/96/8a/f90c59ed25d75e50a8c10a1b1c2d4c402e4dacfa87f3aff36a
Successfully built langdetect


## Google api for translating 

In [42]:
!pip install itranslate

     |████████████████████████████████| 76 kB 651 kB/s            
     |████████████████████████████████| 58 kB 1.8 MB/s            
     |████████████████████████████████| 54 kB 1.4 MB/s            
  Attempting uninstall: h11
    Found existing installation: h11 0.13.0
    Uninstalling h11-0.13.0:
      Successfully uninstalled h11-0.13.0


In [26]:
#!pip install google_trans_new

In [77]:
x, y = data['adDescription'][1], data['adLangDetected'][1]
x, y

('Die Highlights:<br />» Gut geschnittener Büroraum in einer gepflegten Liegenschaft<br />» Ansprechende Büroraum mit 145 m2 <br />» WC-Anlage zur Mitbenutzung <br />» Verkehrsgünstige Lage <br />» Hervorragender Anschluss an den öffentlichen Verkehr<br /><br />Die zusätzlichen Möglichkeiten:<br />» Einstellplätze (à CHF 180.00 pro Monat)<br /><br />Die Lage:<br />Die Liegenschaft befindet sich im Kreis 7 in Zürich und bietet Ihnen alle Annehmlichkeiten einer Grossstadt. Hervorzuheben sind die vielfältigen kulturellen Einrichtungen und die Möglichkeit, zahlreichen Freizeitaktivitäten nachzugehen. <br /><br />Die Infrastruktur:<br />In unmittelbarer Nähe befinden sich Einkaufsmöglichkeiten (Denner, Coop, etc.), Restaurants, Detailhändler und das Schulhaus Langmatt (Primarschule & Kindergarten). Zum Zürichsee sind es weniger als 2.5 km.<br /><br />Die Verkehrsanbindung:<br />Direkt vor der Türe finden Sie eine Bushaltestelle. Mit der Tram 11 gelangen Sie von der Haltestelle Balgrist rasc

In [57]:
#!pip install -U deep-translator

In [79]:
from deep_translator import GoogleTranslator
#translated = GoogleTranslator(source=y, target='en')
#translate = translated.translate(x)
#translate

'The highlights:<br />» Well-designed office space in a well-kept property<br />» Attractive office space with 145 m2 <br />» Toilet facility for shared use <br />» Convenient location <br />» Excellent connection to public transport<br /><br />The additional options:<br />» Parking spaces (at CHF 180.00 per month)<br /><br />The location:<br />The property is in District 7 in Zurich and offers you all the amenities of a big city. The diverse cultural facilities and the opportunity to pursue numerous leisure activities should be emphasized. <br /><br />The infrastructure:<br />In the immediate vicinity there are shops (Denner, Coop, etc.), restaurants, retailers and the Langmatt school (primary school and kindergarten). It is less than 2.5 km to Lake Zurich.<br /><br />Transportation:<br />There is a bus stop right in front of the door. Tram 11 takes you quickly from the Balgrist stop to the center of Zurich. The nearest motorway connection to the A52 is about 4 km away. It is around 1

In [80]:
def translate(text, langDetect):
    trans = GoogleTranslator(source=langDetect, target='en').translate(text)
    return trans

In [81]:
translate(x,y)

'The highlights:<br />» Well-designed office space in a well-kept property<br />» Attractive office space with 145 m2 <br />» Toilet facility for shared use <br />» Convenient location <br />» Excellent connection to public transport<br /><br />The additional options:<br />» Parking spaces (at CHF 180.00 per month)<br /><br />The location:<br />The property is in District 7 in Zurich and offers you all the amenities of a big city. The diverse cultural facilities and the opportunity to pursue numerous leisure activities should be emphasized. <br /><br />The infrastructure:<br />In the immediate vicinity there are shops (Denner, Coop, etc.), restaurants, retailers and the Langmatt school (primary school and kindergarten). It is less than 2.5 km to Lake Zurich.<br /><br />Transportation:<br />There is a bus stop right in front of the door. Tram 11 takes you quickly from the Balgrist stop to the center of Zurich. The nearest motorway connection to the A52 is about 4 km away. It is around 1